In [14]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import wbgapi as wb

In [15]:
data = wb.series.info(q="Government expenditure on education, total")
print(data)

id                 value
-----------------  ------------------------------------------------------------------------
SE.XPD.TOTL.GB.ZS  Government expenditure on education, total (% of government expenditure)
SE.XPD.TOTL.GD.ZS  Government expenditure on education, total (% of GDP)
                   2 elements


In [16]:
# independent variables for the model
inflation_data_id = "FP.CPI.TOTL.ZG"
gfcf_data_id = "NE.GDI.FTOT.ZS"
gov_exp_data_id = "SE.XPD.TOTL.GB.ZS"
life_expectancy_data_id = "SP.DYN.LE00.IN"

# dependent variable for the model
gdp_per_capita_data_id = "NY.GDP.PCAP.PP.CD"

In [17]:
european_country_codes = [
    "ALB", "AND", "AUT", "BEL", "BGR", "BIH", "BLR", "CHE", "CYP", "CZE", "DEU", 
    "DNK", "ESP", "EST", "FIN", "FRA", "GBR", "GEO", "GRC", "HRV", "HUN", "IRL", 
    "ISL", "ITA", "KAZ", "KGZ", "XKX", "LVA", "LIE", "LTU", "LUX", "MCO", "MDA", 
    "MKD", "MLT", "MNE", "NLD", "NOR", "POL", "PRT", "ROU", "RUS", "SMR", "SRB", 
    "SVK", "SVN", "SWE", "TUR", "UKR", "VAT"
]


In [18]:
# let's now fetch the data
inflation_data = wb.data.DataFrame(inflation_data_id, time=range(2000, 2020), labels=True)
gfcf_data = wb.data.DataFrame(gfcf_data_id, time=range(2000, 2020), labels=True)
gov_exp_data = wb.data.DataFrame(gov_exp_data_id, time=range(2000, 2020), labels=True)
life_expectancy_data = wb.data.DataFrame(life_expectancy_data_id, time=range(2000, 2020), labels=True)
gdp_per_capita_data = wb.data.DataFrame(gdp_per_capita_data_id, time=range(2000, 2020), labels=True)


# Preparation of every dataset then merging

In [19]:
gfcf_data = gfcf_data.reset_index()
gfcf_data = gfcf_data[gfcf_data["economy"].isin(european_country_codes)]
gfcf_data = gfcf_data.rename(columns={"economy": "country_code"})
df_long_gfcf = pd.melt(gfcf_data, id_vars=['country_code', 'Country'], var_name='Year', value_name='GFCF')

inflation_data = inflation_data.reset_index()
inflation_data = inflation_data[inflation_data["economy"].isin(european_country_codes)]
inflation_data = inflation_data.rename(columns={"economy": "country_code"})
df_long_inflation = pd.melt(inflation_data, id_vars=['country_code', 'Country'], var_name='Year', value_name='Inflation')

gov_exp_data = gov_exp_data.reset_index()
gov_exp_data = gov_exp_data[gov_exp_data["economy"].isin(european_country_codes)]
gov_exp_data = gov_exp_data.rename(columns={"economy": "country_code"})
df_long_gov_exp = pd.melt(gov_exp_data, id_vars=['country_code', 'Country'], var_name='Year', value_name='Government Expenditure on Education')


life_expectancy_data = life_expectancy_data.reset_index()
life_expectancy_data = life_expectancy_data[life_expectancy_data["economy"].isin(european_country_codes)]
life_expectancy_data = life_expectancy_data.rename(columns={"economy": "country_code"})
df_long_life_expectancy = pd.melt(life_expectancy_data, id_vars=['country_code', 'Country'], var_name='Year', value_name='Life Expectancy')

gdp_per_capita_data = gdp_per_capita_data.reset_index()
gdp_per_capita_data = gdp_per_capita_data[gdp_per_capita_data["economy"].isin(european_country_codes)]
gdp_per_capita_data = gdp_per_capita_data.rename(columns={"economy": "country_code"})
df_long_gdp_per_capita = pd.melt(gdp_per_capita_data, id_vars=['country_code', 'Country'], var_name='Year', value_name='GDP per Capita')

In [20]:
# let's merge all the dataframes
df_long = df_long_inflation.merge(df_long_gfcf, on=['country_code', 'Country', 'Year'])
df_long = df_long.merge(df_long_gov_exp, on=['country_code', 'Country', 'Year'])
df_long = df_long.merge(df_long_life_expectancy, on=['country_code', 'Country', 'Year'])
df_long = df_long.merge(df_long_gdp_per_capita, on=['country_code', 'Country', 'Year'])

df_long


,country_code,Country,Year,Inflation,GFCF,Government Expenditure on Education,Life Expectancy,GDP per Capita
0,GBR,United Kingdom,YR2000,1.182956,17.809370,NaN,77.741463,26529.852173
1,UKR,Ukraine,YR2000,28.203097,19.744731,NaN,67.675610,4260.276855
2,TUR,Turkiye,YR2000,54.915371,22.211352,NaN,71.861000,9498.964414
3,CHE,Switzerland,YR2000,1.558529,27.512043,NaN,79.680488,36695.207055
4,SWE,Sweden,YR2000,0.899144,22.259817,NaN,79.643902,29622.038475
...,...,...,...,...,...,...,...,...
975,BEL,Belgium,YR2019,1.436820,24.283451,11.817001,81.995122,56621.541761
976,BLR,Belarus,YR2019,5.598156,27.034409,13.078263,74.226829,22301.576689
977,AUT,Austria,YR2019,1.530896,24.914393,9.881763,81.895122,60574.627103
978,AND,Andorra,YR2019,NaN,NaN,10.895240,NaN,57573.693049


In [21]:
# Renaming the columns
df_long = df_long.rename(columns={"Country": "country", "Year": "year", "Government Expenditure on Education": "gov_exp", "GDP per Capita": "gdp_per_capita", "Life Expectancy": "life_expectancy", "Inflation": "inflation", "GFCF": "gfcf"})

In [22]:
# let's drop rows that have missing values
df_long = df_long.dropna()
df_long.isnull().sum()

country_code       0
country            0
year               0
inflation          0
gfcf               0
gov_exp            0
life_expectancy    0
gdp_per_capita     0
dtype: int64

# Remove outliers

In [23]:
# Calculate the mean and standard deviation
mean_val = df_long['gfcf'].mean()
std_dev = df_long['gfcf'].std()

# Define a function to identify outliers based on z-score
def remove_outliers(df, column_name, z_thresh=2.5):
    z_scores = np.abs((df[column_name] - mean_val) / std_dev)
    df_cleaned = df[z_scores < z_thresh]
    return df_cleaned

# Remove outliers from the DataFrame
df_cleaned = remove_outliers(df_long, 'gfcf')

# Check the shape of the cleaned DataFrame to see how many outliers were removed
print("Original DataFrame shape:", df_long.shape)
print("Cleaned DataFrame shape:", df_cleaned.shape)


Original DataFrame shape: (588, 8)
Cleaned DataFrame shape: (573, 8)


In [24]:
# let's write out to a csv file
df_cleaned.to_csv("european_countries_data.csv", index=False)